In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default() 
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
!ls

adc.json  drive  sample_data


In [0]:
import os
os.chdir('drive/Brain-Tumor-Segmentation-using-Deep-Neural-networks-master')

In [6]:
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

Using TensorFlow backend.


In [7]:
m = keras.models.load_model('trial_MFCcascade_acc.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [8]:
!pip3 install SimpleITK

     |████████████████████████████████| 42.5MB 35.7MB/s 


In [0]:
import SimpleITK as sitk
import numpy as np

In [15]:
  path = 'HG/0002'
  p = os.listdir(path)
  p.sort(key=str.lower)
  arr = []
  for i in range(len(p)):
    if(i != 4):
      p1 = os.listdir(path+'/'+p[i])
      p1.sort()
      img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
      arr.append(sitk.GetArrayFromImage(img))
    else:
      p1 = os.listdir(path+'/'+p[i])
      img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
      Y_labels = sitk.GetArrayFromImage(img)    
  data = np.zeros((Y_labels.shape[1],Y_labels.shape[0],Y_labels.shape[2],4))
  for i in range(Y_labels.shape[1]):
    data[i,:,:,0] = arr[0][:,i,:]
    data[i,:,:,1] = arr[1][:,i,:]
    data[i,:,:,2] = arr[2][:,i,:]
    data[i,:,:,3] = arr[3][:,i,:]
    info = []
    for i in range(100,data.shape[0],2):
      d = data_gen(data,Y_labels,i,3)
      if(len(d) != 0):
        y = np.zeros((d[2].shape[0],1,1,5))
        for j in range(y.shape[0]):
          y[j,:,:,d[2][j]] = 1
        X1 = d[0]
        X2 = d[1]
        class_weights = class_weight.compute_class_weight('balanced',
                                                        np.unique(d[2]),
                                                        d[2])
        print('slice no:'+str(i)) 
        info.append(m.fit([X1,X2],y,epochs=2,batch_size=256,class_weight= class_weights))
        m.save('trial_MFCcascade_acc.h5')

slice no:100
Epoch 1/2
13392/13392 [==============================] - 69s 5ms/step - loss: 0.1502 - acc: 0.9824
Epoch 2/2
13392/13392 [==============================] - 56s 4ms/step - loss: 0.0787 - acc: 1.0000
slice no:100
Epoch 1/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0700 - acc: 1.0000
Epoch 2/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0640 - acc: 1.0000
slice no:100
Epoch 1/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0589 - acc: 1.0000
Epoch 2/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0545 - acc: 1.0000
slice no:102
Epoch 1/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0514 - acc: 1.0000
Epoch 2/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0478 - acc: 1.0000
slice no:100
Epoch 1/2
13392/13392 [==============================] - 57s 4ms/step - loss: 0.0449 - acc: 1.0000
Epoch 2/2
13392/13392 [=====================

KeyboardInterrupt: ignored

In [0]:
def model_gen(input_dim,x,y,slice_no):
  X1 = []
  X2 = []
  Y = []
  
  for i in range(int((input_dim)/2),y.shape[0]-int((input_dim)/2)):
    for j in range(int((input_dim)/2),y.shape[2]-int((input_dim)/2)):
      if(x[i-16:i+17,j-16:j+17,:].any != 0):
        X2.append(x[i-16:i+17,j-16:j+17,:])
        X1.append(x[i-int((input_dim)/2):i+int((input_dim)/2)+1,j-int((input_dim)/2):j+int((input_dim)/2)+1,:])
        Y.append(y[i,slice_no,j])
      
      
  X1 = np.asarray(X1)
  X2 = np.asarray(X2)
  Y = np.asarray(Y)
  d = [X1,X2,Y]
  return d

In [0]:
def data_gen(data,y,slice_no,model_no):
  d = []
  x = data[slice_no]
  if(x.any() != 0 and y.any() != 0):
    if(model_no == 0):
      X1 = []
      for i in range(16,159):
        for j in range(16,199):
          if(x[i-16:i+17,j-16:j+17,:].all != 0):
            X1.append(x[i-16:i+17,j-16:j+17,:])
      Y1 = []
      for i in range(16,159):
        for j in range(16,199):
          if(x[i-16:i+17,j-16:j+17,:].all != 0):
            Y1.append(y[i,slice_no,j]) 
      X1 = np.asarray(X1)
      Y1 = np.asarray(Y1)
      d = [X1,Y1]
    elif(model_no == 1):
      d = model_gen(65,x,y,slice_no)
    elif(model_no == 2):
      d = model_gen(56,x,y,slice_no)
    elif(model_no == 3):
      d = model_gen(53,x,y,slice_no)  
    
  return d   

In [0]:
from sklearn.utils import class_weight

In [17]:
info = []
for i in range(0,data.shape[0]):
  d = data_gen(data,Y_labels,i,3)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(m.evaluate([X1,X2],y,batch_size=256))
    m.save('trial_0001_MFCcas_dim2_128_acc.h5')

slice no:40
13392/13392 [==============================] - 18s 1ms/step
slice no:41
13392/13392 [==============================] - 19s 1ms/step
slice no:42
13392/13392 [==============================] - 18s 1ms/step
slice no:43
13392/13392 [==============================] - 19s 1ms/step
slice no:44
13392/13392 [==============================] - 18s 1ms/step
slice no:45
13392/13392 [==============================] - 19s 1ms/step
slice no:46
13392/13392 [==============================] - 18s 1ms/step
slice no:47
13392/13392 [==============================] - 19s 1ms/step
slice no:48
13392/13392 [==============================] - 19s 1ms/step
slice no:49
13392/13392 [==============================] - 19s 1ms/step
slice no:50
13392/13392 [==============================] - 19s 1ms/step
slice no:51
13392/13392 [==============================] - 19s 1ms/step
slice no:52
13392/13392 [==============================] - 19s 1ms/step
slice no:53
13392/13392 [==============================] - 19s 1

KeyboardInterrupt: ignored

In [0]:
import h5py
hf = h5py.File('info_mfc.h5', 'w')
hf.create_dataset('dataset_1', data=info)
hf.close()

In [29]:
m1 = keras.models.load_model('trial_0001_input_cascasde_acc.h5')

OSError: ignored

In [0]:
info1 = []
for i in range(0,data.shape[0]):
  d = data_gen(data,Y_labels,i,1)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info1.append(m1.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
info1

In [0]:
import h5py
hf = h5py.File('info1_input.h5', 'w')
hf.create_dataset('dataset_1', data=info1)
hf.close()

In [0]:
info = []
for i in range(100,data.shape[0],2):
  d = data_gen(data,Y_labels,i,3)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(m.fit([X1,X2],y,epochs=2,batch_size=256,class_weight= class_weights))
    m.save('trial_MFCcascade_acc.h5')

In [0]:
len(info)

InputCascadeCNN training

In [20]:
m1 = keras.models.load_model('trial_0001_input_cascasde_acc.h5')

OSError: ignored

In [21]:
fold = os.listdir('HG/')
fold.sort(key=str.lower) 

for path in fold[4:-1]:
    print(path)
    path = 'HG/'+path
    p = os.listdir(path)
    p.sort(key=str.lower)
    arr = []
    for i in range(len(p)):
      if(i != 4):
        p1 = os.listdir(path+'/'+p[i])
        p1.sort()
        img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
        arr.append(sitk.GetArrayFromImage(img))
      else:
        p1 = os.listdir(path+'/'+p[i])
        img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
        Y_labels = sitk.GetArrayFromImage(img)
    data = np.zeros((Y_labels.shape[1],Y_labels.shape[0],Y_labels.shape[2],4))
    for i in range(Y_labels.shape[1]):
      data[i,:,:,0] = arr[0][:,i,:]
      data[i,:,:,1] = arr[1][:,i,:]
      data[i,:,:,2] = arr[2][:,i,:]
      data[i,:,:,3] = arr[3][:,i,:]
    print(data.shape)
    info = []
    for i in range(154,data.shape[0],2):
      d = data_gen(data,Y_labels,i,1)
      if(len(d) != 0):
        y = np.zeros((d[2].shape[0],1,1,5))
        for j in range(y.shape[0]):
          y[j,:,:,d[2][j]] = 1
        X1 = d[0]
        X2 = d[1]
        class_weights = class_weight.compute_class_weight('balanced',
                                                          np.unique(d[2]),
                                                          d[2])
        print('slice no:'+str(i))
        info.append(m1.fit([X1,X2],y,epochs=5,batch_size=128,class_weight= class_weights))
        m1.save('trial_0001_input_cascasde_acc.h5')


0005
(216, 176, 160, 4)
slice no:154


NameError: ignored

In [22]:
print(i)

154


simple 2path CNN training

In [26]:
m0 = keras.models.load_model('trial_0001_2path_acc.h5')

OSError: ignored

In [0]:
m0.summary()

In [25]:
info = []
for i in range(100,data.shape[0],2):
  d = data_gen(data,Y_labels,i,0)
  if(len(d) != 0):
    y = np.zeros((d[-1].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[-1][j]] = 1
    sample = np.zeros((5,1))
    for j in range(5):
      sample[j] = np.sum(y[:,:,:,j])
    print(np.transpose(sample/np.sum(sample)))    
    X1 = d[0]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[-1]),
                                                      d[-1])
    print('slice no:'+str(i))
    info.append(m0.fit(X1,y,epochs=2,batch_size=1024,class_weight= class_weights))
    m0.save('trial_0001_2path_acc.h5')

IndexError: ignored

In [0]:
import keras

In [0]:
model = keras.models.load_model('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
info = []
for i in range(0,data.shape[0],2):
  d = data_gen(data,Y_labels,i,3)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(model.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
m = keras.models.load_model('trial_0001_input_cascasde_acc.h5')

In [0]:
info = []
for i in range(0,data.shape[0],2):
  d = data_gen(data,Y_labels,i,1)
  if(len(d) != 0):
    y = np.zeros((d[2].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[2][j]] = 1
    X1 = d[0]
    X2 = d[1]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[2]),
                                                      d[2])
    print('slice no:'+str(i))
    info.append(m.evaluate([X1,X2],y,batch_size=256))
    #model.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
hf = h5py.File('info_mfc.h5', 'r')
info = hf.get('dataset_1')

In [0]:
info = np.asarray(info)

In [24]:
info

[]

In [0]:
info1

array([[4.50206129e-05, 1.00000000e+00],
       [4.57702571e-05, 1.00000000e+00],
       [4.60857943e-05, 1.00000000e+00],
       [4.84986886e-05, 1.00000000e+00],
       [4.89885355e-05, 1.00000000e+00],
       [5.29922178e-05, 1.00000000e+00],
       [6.21655353e-05, 1.00000000e+00],
       [8.30534716e-05, 1.00000000e+00],
       [1.11175580e-04, 1.00000000e+00],
       [1.85727202e-04, 1.00000000e+00],
       [2.48078846e-04, 1.00000000e+00],
       [5.93782379e-04, 1.00000000e+00],
       [6.89002551e-04, 1.00000000e+00],
       [9.09570965e-04, 1.00000000e+00],
       [8.90290332e-04, 1.00000000e+00],
       [9.80984229e-04, 1.00000000e+00],
       [9.21154452e-04, 1.00000000e+00],
       [9.82793154e-04, 1.00000000e+00],
       [1.00479382e-03, 1.00000000e+00],
       [1.12422367e-03, 1.00000000e+00],
       [1.23042761e-03, 1.00000000e+00],
       [1.32213331e-03, 1.00000000e+00],
       [1.31603022e-03, 1.00000000e+00],
       [1.38255507e-03, 1.00000000e+00],
       [1.378436

In [0]:
info1-info

array([[-2.42181185e-03,  0.00000000e+00],
       [-2.40587245e-03,  0.00000000e+00],
       [-2.39603168e-03,  0.00000000e+00],
       [-2.39542873e-03,  0.00000000e+00],
       [-2.39141447e-03,  0.00000000e+00],
       [-2.41818763e-03,  0.00000000e+00],
       [-2.45722313e-03,  0.00000000e+00],
       [-2.70171757e-03,  0.00000000e+00],
       [-3.20715355e-03,  0.00000000e+00],
       [-3.91375285e-03,  0.00000000e+00],
       [-3.59821219e-03,  0.00000000e+00],
       [-1.27869890e-02,  3.65890084e-03],
       [-1.16920460e-02,  3.50955795e-03],
       [-1.01826541e-02,  2.83751493e-03],
       [-9.17154339e-03,  2.38948626e-03],
       [-1.00100991e-02,  2.68817204e-03],
       [-7.75346538e-03,  1.86678614e-03],
       [-5.92540888e-03,  1.19474313e-03],
       [-5.12973246e-03,  8.21385902e-04],
       [-5.82547535e-03,  1.41875747e-03],
       [-3.80339624e-03,  4.48028674e-04],
       [-2.51007749e-03,  0.00000000e+00],
       [-2.18508295e-03,  0.00000000e+00],
       [-2.